# <div align="center" style="color: #ff5733;">Data Requested By Annastasia for Monthly Reporting</div>

In [1]:
# %% [markdown]
# # Jupyter Notebook Loading Header
#
# This is a custom loading header for Jupyter Notebooks in Visual Studio Code.
# It includes common imports and settings to get you started quickly.

# %% [markdown]
## Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
import os
path = r'C:\Users\DwaipayanChakroborti\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path
client = bigquery.Client(project='prj-prod-dataplatform')

# %% [markdown]
## Configure Settings
# Set options or configurations as needed
# Example: pd.set_option('display.max_columns', None)

In [2]:
# Year and Month for which data is extracted
data_year = 2024
data_month = 8

In [3]:
# How many Applications applied to each month

sq = """
# loan applied
SELECT 
    DATE_TRUNC( startApplyDateTime,MONTH) as mm, 
    case when reloan_flag = 1  and loantype not like 'FLEXUP' then 'Reloan'
         when loantype = 'FLEXUP' and new_loan_type = 'Flex-up' and reloan_flag = 0 and flagDisbursement = 1 then 'Flex-up' 
         
         else new_loan_type end as LoanProduct,
    count (distinct digitalLoanAccountId) as StartedApps
FROM 
    `prj-prod-dataplatform.risk_credit_mis.loan_master_table` 

group by 1,2
order by 1 desc,2;
"""

StartedApps = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The rows and columns in StartedApps are:\t {StartedApps.shape}")

Job ID 7094a058-822a-4912-8fb2-02231a26ddd9 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The rows and columns in StartedApps are:	 (202, 3)


In [4]:
# How many submitted applications to each month
sq = """
# loan Submitted
SELECT 
    DATE_TRUNC((case when loantype = 'FLEXUP' and new_loan_type = 'Flex-up' and reloan_flag = 0 and flagDisbursement = 1 then startApplyDateTime else termsAndConditionsSubmitDateTime end),MONTH) as mm,  
    case when reloan_flag = 1 and loantype not like 'FLEXUP'then 'Reloan'
         when loantype = 'FLEXUP' and new_loan_type = 'Flex-up' and reloan_flag = 0 and flagDisbursement = 1 then 'Flex-up' 
                  else new_loan_type end as LoanProduct,
    count (distinct digitalLoanAccountId) as SubmittedApps
FROM 
    `prj-prod-dataplatform.risk_credit_mis.loan_master_table` 
group by 1,2
order by 1 desc,2;
"""

SubmittedApps = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The rows and columns of SubmittedApps are:\t {SubmittedApps.shape}")
SubmittedApps.sample(10)

Job ID c2878deb-e548-49f2-b58b-9e074cbe9d0c successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The rows and columns of SubmittedApps are:	 (202, 3)


,mm,LoanProduct,SubmittedApps
13,2024-07-01,Flex,5
27,2024-05-01,Quick,24636
138,2022-10-01,Quick,34284
77,2023-09-01,SIL-Instore,5598
4,2024-09-01,SIL-Instore,2292
119,2023-02-01,SIL-Instore,981
19,2024-06-01,Flex,10
39,2024-03-01,Quick,17988
133,2022-11-01,Quick,20077
11,2024-08-01,SIL-Instore,24167


In [5]:
# Approved Loans for each month

sq = """
# Approved loans
SELECT 
DATE_TRUNC((case when loantype = 'FLEXUP' and new_loan_type = 'Flex-up' and reloan_flag = 0 and flagDisbursement = 1 then startApplyDateTime 
                  when reloan_flag = 1 and loantype not like 'FLEXUP' then startApplyDateTime else decision_date end),MONTH) as mm,
        case when reloan_flag = 1 and loantype not like 'FLEXUP'then 'Reloan'
         when loantype = 'FLEXUP' and new_loan_type = 'Flex-up' and reloan_flag = 0 and flagDisbursement = 1 then 'Flex-up' 
                  else new_loan_type end as LoanProduct,
    count (distinct digitalLoanAccountId) as ApprovedApps
FROM 
    `prj-prod-dataplatform.risk_credit_mis.loan_master_table`
where 
(case when loantype = 'FLEXUP' and new_loan_type = 'Flex-up' and reloan_flag = 0 and flagDisbursement = 1 then flagDisbursement 
      when reloan_flag = 1 and loantype not like 'FLEXUP' then flagDisbursement  else flagApproval end) = 1
group by 1,2
order by 1 desc,2
;
"""

ApprovedApps = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The rows and columns in ApprovedApps are:\t {ApprovedApps.shape}")
ApprovedApps.head(10)

Job ID 9762d68a-8e92-428f-9d70-b271b67b6e61 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The rows and columns in ApprovedApps are:	 (164, 3)


,mm,LoanProduct,ApprovedApps
0,2024-09-01,Flex-up,39
1,2024-09-01,Quick,369
2,2024-09-01,Reloan,39
3,2024-09-01,SIL ZERO,161
4,2024-09-01,SIL-Instore,1402
5,2024-08-01,Flex-up,602
6,2024-08-01,Quick,3439
7,2024-08-01,Reloan,427
8,2024-08-01,SIL ZERO,440
9,2024-08-01,SIL-Instore,14663


In [6]:
# Booked Loans for each month

sq = """
#Booked loans
SELECT 
DATE_TRUNC( disbursementDateTime,MONTH) as mm, 
            case when reloan_flag = 1 and loantype not like 'FLEXUP' and flagDisbursement = 1 then 'Reloan'
         when loantype = 'FLEXUP' and new_loan_type = 'Flex-up' and reloan_flag = 0 and flagDisbursement = 1 then 'Flex-up' 
                  else new_loan_type end as LoanProduct,
    count (distinct digitalLoanAccountId) as BookedApps
FROM 
    `prj-prod-dataplatform.risk_credit_mis.loan_master_table` 
    where flagDisbursement = 1

group by 1,2
order by 1 desc,2
;
"""

BookedApps = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The rows and columns in BookedApps are:\t {BookedApps.shape}")
BookedApps.head(10)

Job ID 1a4d30a3-f037-4684-9687-b9b85d6b884b successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The rows and columns in BookedApps are:	 (163, 3)


,mm,LoanProduct,BookedApps
0,2024-09-01,Flex-up,67
1,2024-09-01,Quick,374
2,2024-09-01,Reloan,39
3,2024-09-01,SIL ZERO,155
4,2024-09-01,SIL-Instore,1383
5,2024-08-01,Flex-up,733
6,2024-08-01,Quick,3289
7,2024-08-01,Reloan,427
8,2024-08-01,SIL ZERO,437
9,2024-08-01,SIL-Instore,14638


In [7]:
# Get the total booked amount for a month

sq = """
# Booked Amount
SELECT 
DATE_TRUNC( disbursementDateTime,MONTH) as mm, 
    case when reloan_flag = 1 and loantype not like 'FLEXUP' and flagDisbursement = 1 then 'Reloan'
         when loantype = 'FLEXUP' and new_loan_type = 'Flex-up' and reloan_flag = 0 and flagDisbursement = 1 then 'Flex-up' 
                  else new_loan_type end as LoanProduct,
    sum (disbursedLoanAmount) as BookedAmt
FROM 
    `prj-prod-dataplatform.risk_credit_mis.loan_master_table` 
     where flagDisbursement = 1
group by 1,2
order by 1 desc,2
;
"""
BookedAmt = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The rows and columns in BookedAmt are:\t {BookedAmt.shape}")
BookedAmt.head(10)

Job ID f05c7079-6183-4632-ac4a-dd5d6cc1b2be successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The rows and columns in BookedAmt are:	 (163, 3)


,mm,LoanProduct,BookedAmt
0,2024-09-01,Flex-up,2.427885e+06
1,2024-09-01,Quick,4.786500e+06
2,2024-09-01,Reloan,1.000178e+06
3,2024-09-01,SIL ZERO,2.223461e+06
4,2024-09-01,SIL-Instore,1.251550e+07
5,2024-08-01,Flex-up,2.369153e+07
6,2024-08-01,Quick,4.151086e+07
7,2024-08-01,Reloan,1.010533e+07
8,2024-08-01,SIL ZERO,6.431638e+06
9,2024-08-01,SIL-Instore,1.419244e+08


In [8]:
BookedAmt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 163 entries, 0 to 162
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   mm           163 non-null    datetime64[us]
 1   LoanProduct  163 non-null    object        
 2   BookedAmt    163 non-null    float64       
dtypes: datetime64[us](1), float64(1), object(1)
memory usage: 3.9+ KB


In [9]:
raw_data = StartedApps.merge(SubmittedApps,how='outer').merge(ApprovedApps,how='outer').merge(BookedApps,how='outer').merge(BookedAmt,how='outer')

In [10]:
raw_data.tail(10)

,mm,LoanProduct,StartedApps,SubmittedApps,ApprovedApps,BookedApps,BookedAmt
194,2024-08-01,Flex-up,602,602,602,733,2.369153e+07
195,2024-08-01,Quick,67342,55491,3439,3289,4.151086e+07
196,2024-08-01,Reloan,470,449,427,427,1.010533e+07
197,2024-08-01,SIL ZERO,994,969,440,437,6.431638e+06
198,2024-08-01,SIL-Instore,24505,24167,14663,14638,1.419244e+08
199,2024-09-01,Flex-up,39,39,39,67,2.427885e+06
200,2024-09-01,Quick,5658,4772,369,374,4.786500e+06
201,2024-09-01,Reloan,48,45,39,39,1.000178e+06
202,2024-09-01,SIL ZERO,316,311,161,155,2.223461e+06
203,2024-09-01,SIL-Instore,2333,2292,1402,1383,1.251550e+07


In [11]:
print(f'{data_year}-{data_month:02d}-01')

2024-08-01


In [12]:
raw_data_current_month = raw_data[(raw_data['mm']>='2022-08-01') &(raw_data['mm']<=f'{data_year}-{data_month:02d}-01')]

In [13]:
raw_data_current_month

,mm,LoanProduct,StartedApps,SubmittedApps,ApprovedApps,BookedApps,BookedAmt
54,2022-08-01,Flex,5558,4732,594,599,1.198500e+07
55,2022-08-01,Quick,43764,30660,1910,1926,3.300100e+07
56,2022-08-01,SIL-Instore,967,956,456,457,1.137941e+07
57,2022-08-01,SIL-Online(discontinued),1192,652,41,41,1.197499e+06
58,2022-09-01,Big Loan,13,12,<NA>,<NA>,NaN
...,...,...,...,...,...,...,...
194,2024-08-01,Flex-up,602,602,602,733,2.369153e+07
195,2024-08-01,Quick,67342,55491,3439,3289,4.151086e+07
196,2024-08-01,Reloan,470,449,427,427,1.010533e+07
197,2024-08-01,SIL ZERO,994,969,440,437,6.431638e+06


### Loan Account Tagging

In [14]:
sq="""
# Loan Tagging
SELECT
    loanAccountNumber,
    Case when reloan_flag = 1 and loantype not like 'FLEXUP' and flagDisbursement = 1 then 'Reloan'
         when loantype = 'FLEXUP' and new_loan_type = 'Flex-up' and reloan_flag = 0 and flagDisbursement = 1 then 'Flex-up' 
                  else new_loan_type end as LoanProduct,
    disbursementDateTime,
    (CASE WHEN new_loan_type = 'Flex-up' then
    LAG(new_loan_type) OVER (PARTITION BY customerId ORDER BY disbursementDateTime) END) AS OriginalLoanProduct
  FROM
    `prj-prod-dataplatform.risk_credit_mis.loan_master_table`
   where flagDisbursement=1
;
"""
loan_type = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The rows and columns in loan_type are: {loan_type.shape}")
loan_type.tail(10)

Job ID 101c615d-3162-4466-9561-5e919cd158c5 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The rows and columns in loan_type are: (188700, 4)


,loanAccountNumber,LoanProduct,disbursementDateTime,OriginalLoanProduct
188690,60828136490016,SIL-Instore,2024-09-01 12:25:12,None
188691,60828138920015,SIL-Instore,2024-09-01 13:19:43,None
188692,60828142140017,SIL-Instore,2024-09-01 14:14:35,None
188693,60828142480017,SIL-Instore,2024-09-01 14:25:31,None
188694,60828146160011,SIL-Instore,2024-09-01 15:37:26,None
188695,60828147340016,SIL-Instore,2024-09-01 16:02:01,None
188696,60828156760017,SIL-Instore,2024-09-01 20:02:02,None
188697,60828178190012,SIL-Instore,2024-09-02 17:23:39,None
188698,60828179850015,SIL-Instore,2024-09-02 18:18:59,None
188699,60828192440013,SIL-Instore,2024-09-03 11:56:31,None


In [15]:
loan_type[loan_type['loanAccountNumber']=='60828136490016']

,loanAccountNumber,LoanProduct,disbursementDateTime,OriginalLoanProduct
188690,60828136490016,SIL-Instore,2024-09-01 12:25:12,None


In [16]:
Loan_Acs_current_month = loan_type[(loan_type['disbursementDateTime'].dt.month == data_month) & (loan_type['disbursementDateTime'].dt.year == data_year)]

In [17]:
Loan_Acs_current_month = Loan_Acs_current_month[['loanAccountNumber', 'LoanProduct','OriginalLoanProduct']]

#### VAS

In [18]:
sq = """
# Vas 
SELECT 
    extract(year from disbursementDateTime) as year , 
    extract(month from disbursementDateTime) as month,
    Case when reloan_flag = 1 and b.loantype not like 'FLEXUP' and flagDisbursement = 1 then 'Reloan'
         when b.loantype = 'FLEXUP' and new_loan_type = 'Flex-up' and reloan_flag = 0 and flagDisbursement = 1 then 'Flex-up' 
                  else new_loan_type end as new_loan_type, 
    count(distinct  b.loanAccountNumber ) AS TotalSold,
   
   count(distinct CASE WHEN vas_flag = 'true' THEN b.loanAccountNumber END) AS SoldWithVAS
     FROM `prj-prod-dataplatform.dl_loans_db_raw.tdbk_digital_loan_application` a join
`prj-prod-dataplatform.risk_credit_mis.loan_master_table` b on a.loanAccountNumber = b.loanAccountNumber
where flagDisbursement = 1
group by 1,2 ,3 
order by 1 desc,2 desc,3
;
"""

vas_df = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The rows and columns in vas_df are:\t {vas_df.shape}")
vas_df.head(10)

Job ID b1630617-f98c-4a0b-bfe4-1245407c84a1 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The rows and columns in vas_df are:	 (163, 5)


,year,month,new_loan_type,TotalSold,SoldWithVAS
0,2024,9,Flex-up,67,0
1,2024,9,Quick,374,222
2,2024,9,Reloan,39,0
3,2024,9,SIL ZERO,155,112
4,2024,9,SIL-Instore,1383,960
5,2024,8,Flex-up,733,0
6,2024,8,Quick,3289,2010
7,2024,8,Reloan,427,0
8,2024,8,SIL ZERO,437,325
9,2024,8,SIL-Instore,14638,9985


In [19]:
vas_df_current_month = vas_df[(vas_df['year']==data_year)& (vas_df['month']==data_month)]

In [20]:
with pd.ExcelWriter(f'loan_accounts_tagging_{data_year}_{data_month:02d}_v1.xlsx') as writer:
    # Write each DataFrame to a specific sheet in the Excel file
    raw_data_current_month.to_excel(writer, sheet_name='raw_data', index=False)
    Loan_Acs_current_month.to_excel(writer, sheet_name='Loan Acs', index=False)
    vas_df_current_month.to_excel(writer, sheet_name='VAS', index=False)